In [34]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [35]:
'''
@params
    df: a dataframe from which the null columns are to be removied
    threshold: the total count of nulls in the column to decide whether to remove that column or not
@returns:
    dataframe with deleted columns based on the threshold from the provided dataframe 'df'
'''
def removeableCols(df,threshold):
    count_null = sum(pd.isnull(df[df.columns]),axis=0)
    return count_null[count_null > threshold].index

In [65]:
import pandas as pd
df = pd.read_csv('dataset/train.csv')
#print(df.dtypes)
#df = pd.get_dummies(df)
#remove columns
removColList = removeableCols(df,50)
df = df.drop(removColList,axis=1)

#split dataframe into two with categorical and continuous
cats = [col for col,x in df.dtypes.items() if x == 'object']
cat_df = df[cats]

cont_df = df.drop(cats,axis=1)
cat_df = cat_df.fillna(cat_df.mode(axis=0))

#create dictionary of columns to their modes for
#categorical data
modemaps = cat_df.mode(axis=0).to_dict()
for k in modemaps:
    modemaps[k] = modemaps[k][0]
#print(modemaps)

#create dictionary of columns to their median for
#continuous data
cont_df = cont_df.fillna(cont_df.median())
medianmaps = cont_df.median(axis=0).to_dict()
#print(medianmaps)

#for k in medianmaps:
 #   medianmaps[k] = medianmaps[k][0]
#print(medianmaps)
cat_df = pd.get_dummies(cat_df)
combined_df = pd.concat((cont_df,cat_df),axis=1)
combined_df.shape

(1460, 251)

In [59]:
test_df = pd.read_csv('dataset/test.csv')
test_df = test_df.drop(removColList,axis=1)

#split dataframe into cont and cat
cat_test_df = test_df[cats]
cont_test_df = test_df.drop(cats,axis=1)
cat_test_df = cat_test_df.fillna(value=modemaps)
cat_test_df = pd.get_dummies(cat_test_df)

cont_test_df = cont_test_df.fillna(value=medianmaps)

combined_test_df = pd.concat((cont_test_df,cat_test_df),axis=1)
combined_test_df.shape

(1459, 235)

In [82]:
#remove the columns from training data which are not present
#test data
not_found_cols = list(set(combined_df.columns)-set(combined_test_df.columns))
y = combined_df['SalePrice']
combined_df = combined_df.drop(not_found_cols,axis=1)
combined_df.shape

(1460, 235)

In [92]:
from sklearn.ensemble import RandomForestRegressor as RFG
from sklearn.ensemble import GradientBoostingRegressor as GBR
rfg = RFG(100)
rfg.fit(combined_df,y)
gbr = GBR()
gbr.fit(combined_df,y)
y_pred = (rfg.predict(combined_test_df) + gbr.predict(combined_test_df))/2
#y_pred = gbr.predict(combined_test_df)

In [93]:
submission = pd.concat([test_df['Id'],pd.Series(y_pred,name='SalePrice')],axis=1)
submission.to_csv('dataset/submission_2.csv',index=False)

In [ ]:
'''
try to run the gradient boosting algorithm with all the strings replaced
with the numbers and then try that out it should definitely work pretty well 
and we should gain some point on that
'''